IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M5\FAULT_M5_5


In [3]:
guasto=1

VOLO m5 FAULT 10%

In [4]:
rcou_m5_fault5 = pd.read_csv("RCOU.csv")
rcou_m5_fault5 = rcou_m5_fault5.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)
rcou_m5_fault5 = rcou_m5_fault5[((rcou_m5_fault5['C9'] >= 1350) & (rcou_m5_fault5['C10'] >= 1350) & (rcou_m5_fault5['C11'] >= 1350) & (rcou_m5_fault5['C12'] >= 1350) & (rcou_m5_fault5['C13'] >= 1350) & (rcou_m5_fault5['C14']>= 1350))]
rcou_m5_fault5=rcou_m5_fault5.reset_index(drop=True)
display(rcou_m5_fault5)

,TimeUS,C9,C10,C11,C12,C13,C14
0,76590156,1376,1364,1365,1375,1371,1370
1,76628157,1359,1426,1417,1370,1383,1404
2,76649076,1399,1414,1450,1360,1442,1369
3,76680796,1469,1379,1365,1480,1413,1438
4,76712365,1370,1510,1507,1375,1440,1449
...,...,...,...,...,...,...,...
4510,129967010,1590,1438,1512,1525,1563,1471
4511,129977954,1583,1423,1507,1510,1561,1451
4512,129991804,1538,1435,1457,1520,1486,1492
4513,130003524,1550,1372,1432,1501,1498,1436


In [5]:
min=rcou_m5_fault5['TimeUS'][0]
max=rcou_m5_fault5['TimeUS'][len(rcou_m5_fault5)-1]
print(max)
print(min)

130015745
76590156


In [6]:
xkf1_m5_fault5 = pd.read_csv("XKF1_0.csv")
xkf1_m5_fault5 = xkf1_m5_fault5.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m5_fault5 = xkf1_m5_fault5[((xkf1_m5_fault5['TimeUS'] >= min) & (xkf1_m5_fault5['TimeUS'] <= max	))]
xkf1_m5_fault5 = xkf1_m5_fault5.reset_index(drop=True)
print(xkf1_m5_fault5)

         TimeUS  Roll  Pitch    Yaw
0      76598120  0.62  -0.26  17.98
1      76615343  0.65  -0.25  17.99
2      76625563  0.56  -0.26  17.98
3      76636156  0.52  -0.31  17.98
4      76647059  0.53  -0.33  17.97
...         ...   ...    ...    ...
4855  129964991 -0.62  -2.45   1.17
4856  129975220 -0.57  -2.47   1.15
4857  129989898 -0.34  -2.45   1.13
4858  130001095 -0.07  -2.44   1.10
4859  130013613  0.06  -2.35   1.07

[4860 rows x 4 columns]


In [7]:
att_m5_fault5 = pd.read_csv("ATT.csv")
att_m5_fault5 = att_m5_fault5.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m5_fault5 = att_m5_fault5[((att_m5_fault5['TimeUS'] >= min) & (att_m5_fault5['TimeUS'] <= max	))]
att_m5_fault5=att_m5_fault5.reset_index(drop=True)
print(att_m5_fault5)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0      76600096     0.07  0.62     -0.03  -0.26   17.98  17.98
1      76616574     0.07  0.65     -0.04  -0.25   17.98  17.99
2      76627514     0.08  0.56     -0.04  -0.26   17.98  17.98
3      76638014     0.09  0.52     -0.04  -0.31   17.98  17.98
4      76648250     0.09  0.53     -0.04  -0.33   17.98  17.97
...         ...      ...   ...       ...    ...     ...    ...
4855  129966271     0.56 -0.62     -0.37  -2.45    5.87   1.17
4856  129977309     0.56 -0.57     -0.32  -2.47    5.87   1.15
4857  129991025     0.56 -0.34     -0.28  -2.45    5.87   1.13
4858  130002833     0.54 -0.07     -0.25  -2.44    5.87   1.10
4859  130014932     0.51  0.06     -0.23  -2.35    5.87   1.07

[4860 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m5_fault5)>len(xkf1_m5_fault5):
    to_add=att_m5_fault5[len(xkf1_m5_fault5):]
    att_m5_fault5=att_m5_fault5[:len(xkf1_m5_fault5)]

if len(xkf1_m5_fault5)>len(att_m5_fault5):
    to_add=xkf1_m5_fault5[len(att_m5_fault5):]

for idx,i in enumerate(att_m5_fault5['Roll']):
    if(xkf1_m5_fault5['Roll'][idx] != i):
        att_m5_fault5['Roll'][idx] = ((att_m5_fault5['Roll'][idx] + xkf1_m5_fault5['Roll'][idx])/2)

for idx,i in enumerate(att_m5_fault5['Pitch']):
    if(xkf1_m5_fault5['Pitch'][idx] != i):
        att_m5_fault5['Pitch'][idx] = ((att_m5_fault5['Pitch'][idx] + xkf1_m5_fault5['Pitch'][idx])/2)

for idx,i in enumerate(att_m5_fault5['Yaw']):
    if(xkf1_m5_fault5['Yaw'][idx] != i):
        att_m5_fault5['Yaw'][idx] = ((att_m5_fault5['Yaw'][idx] + xkf1_m5_fault5['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m5_fault5=pd.concat([att_m5_fault5,to_add])

print(att_m5_fault5)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0      76600096     0.07  0.62     -0.03  -0.26   17.98  17.98
1      76616574     0.07  0.65     -0.04  -0.25   17.98  17.99
2      76627514     0.08  0.56     -0.04  -0.26   17.98  17.98
3      76638014     0.09  0.52     -0.04  -0.31   17.98  17.98
4      76648250     0.09  0.53     -0.04  -0.33   17.98  17.97
...         ...      ...   ...       ...    ...     ...    ...
4855  129966271     0.56 -0.62     -0.37  -2.45    5.87   1.17
4856  129977309     0.56 -0.57     -0.32  -2.47    5.87   1.15
4857  129991025     0.56 -0.34     -0.28  -2.45    5.87   1.13
4858  130002833     0.54 -0.07     -0.25  -2.44    5.87   1.10
4859  130014932     0.51  0.06     -0.23  -2.35    5.87   1.07

[4860 rows x 7 columns]


C:\Users\mgale\AppData\Local\Temp\ipykernel_14484\1419169348.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m5_fault5['Yaw'][idx] = ((att_m5_fault5['Yaw'][idx] + xkf1_m5_fault5['Yaw'][idx])/2)


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m5_fault5 = pd.read_csv("ESC_0.csv")
esc_0_m5_fault5 = esc_0_m5_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m5_fault5 = esc_0_m5_fault5[((esc_0_m5_fault5['TimeUS'] >= min) & (esc_0_m5_fault5['TimeUS'] <= max))]
esc_0_m5_fault5=esc_0_m5_fault5.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m5_fault5 = pd.read_csv("ESC_1.csv")
esc_1_m5_fault5 = esc_1_m5_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m5_fault5 = esc_1_m5_fault5[((esc_1_m5_fault5['TimeUS'] >= min) & (esc_1_m5_fault5['TimeUS'] <= max))]
esc_1_m5_fault5=esc_1_m5_fault5.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m5_fault5 = pd.read_csv("ESC_2.csv")
esc_2_m5_fault5 = esc_2_m5_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m5_fault5 = esc_2_m5_fault5[((esc_2_m5_fault5['TimeUS'] >= min) & (esc_2_m5_fault5['TimeUS'] <= max))]
esc_2_m5_fault5=esc_2_m5_fault5.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m5_fault5 = pd.read_csv("ESC_3.csv")
esc_3_m5_fault5 = esc_3_m5_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m5_fault5 = esc_3_m5_fault5[((esc_3_m5_fault5['TimeUS'] >= min) & (esc_3_m5_fault5['TimeUS'] <= max))]
esc_3_m5_fault5=esc_3_m5_fault5.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m5_fault5 = pd.read_csv("ESC_4.csv")
esc_4_m5_fault5 = esc_4_m5_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m5_fault5 = esc_4_m5_fault5[((esc_4_m5_fault5['TimeUS'] >= min) & (esc_4_m5_fault5['TimeUS'] <= max))]
esc_4_m5_fault5=esc_4_m5_fault5.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m5_fault5 = pd.read_csv("ESC_5.csv")
esc_5_m5_fault5 = esc_5_m5_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m5_fault5 = esc_5_m5_fault5[((esc_5_m5_fault5['TimeUS'] >= min) & (esc_5_m5_fault5['TimeUS'] <= max))]
esc_5_m5_fault5=esc_5_m5_fault5.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m5_fault5 = pd.read_csv("IMU_0.csv")
imu_0_m5_fault5 = imu_0_m5_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m5_fault5 = imu_0_m5_fault5[((imu_0_m5_fault5['TimeUS'] >= min) & (imu_0_m5_fault5['TimeUS'] <= max))]

imu_1_m5_fault5 = pd.read_csv("IMU_1.csv")
imu_1_m5_fault5 = imu_1_m5_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m5_fault5 = imu_1_m5_fault5[((imu_1_m5_fault5['TimeUS'] >= min) & (imu_1_m5_fault5['TimeUS'] <= max))]

imu_2_m5_fault5 = pd.read_csv("IMU_2.csv")
imu_2_m5_fault5 = imu_2_m5_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m5_fault5 = imu_2_m5_fault5[((imu_2_m5_fault5['TimeUS'] >= min) & (imu_2_m5_fault5['TimeUS'] <= max))]

imu_m5_fault5 = pd.concat((imu_0_m5_fault5, imu_1_m5_fault5, imu_2_m5_fault5))
imu_m5_fault5=imu_m5_fault5.groupby(imu_m5_fault5.TimeUS, as_index=False).mean()

display(imu_m5_fault5)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,76591750,0.173409,0.043199,0.004685,-0.176615,0.117010,-10.015199
1,76594395,0.076037,-0.019434,-0.004237,-0.040051,-0.149500,-9.643477
2,76597003,-0.153429,-0.056478,-0.003201,0.009088,-0.251848,-9.482471
3,76599933,-0.206979,-0.032131,0.004984,-0.091977,-0.123594,-9.647797
4,76603109,0.003761,0.014892,0.016421,-0.148840,0.052601,-10.130823
...,...,...,...,...,...,...,...
18452,129996960,0.344133,0.040225,-0.040295,-0.996638,0.970077,-13.997997
18453,129999994,0.342809,0.087223,-0.039791,-1.069123,1.056487,-14.131667
18454,130002550,0.355289,0.060848,-0.045159,-1.138574,0.975725,-14.337180
18455,130008951,0.368710,0.092210,-0.050912,-1.847238,0.714070,-15.517767


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m5_fault5 = pd.merge_ordered(imu_m5_fault5,att_m5_fault5)
m5_fault5=m5_fault5.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m5_fault5 = pd.merge_ordered(m5_fault5, esc_0_m5_fault5)
m5_fault5 = pd.merge_ordered(m5_fault5, esc_1_m5_fault5)
m5_fault5 = pd.merge_ordered(m5_fault5, esc_2_m5_fault5)
m5_fault5 = pd.merge_ordered(m5_fault5, esc_3_m5_fault5)
m5_fault5 = pd.merge_ordered(m5_fault5, esc_4_m5_fault5)
m5_fault5 = pd.merge_ordered(m5_fault5, esc_5_m5_fault5)
m5_fault5=m5_fault5.fillna(method="ffill").fillna(method="bfill")
m5_fault5 = pd.merge_ordered(m5_fault5, rcou_m5_fault5)
m5_fault5=m5_fault5.fillna(method="ffill").fillna(method="bfill")

In [12]:
index_to_remove=[]
for ind in att_m5_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_0_m5_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_1_m5_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_2_m5_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_3_m5_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_4_m5_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_5_m5_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in rcou_m5_fault5["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))

4860
5316
5769
6224
6663
7111
7550


In [13]:
m5_fault5 = m5_fault5[~m5_fault5.TimeUS.isin(index_to_remove)]
m5_fault5["TimeUS"] = m5_fault5["TimeUS"] - m5_fault5.iloc[0]["TimeUS"]
m5_fault5["TimeUS"] = m5_fault5["TimeUS"].astype(int)
m5_fault5["Guasto"] = guasto

In [14]:
from datetime import timedelta
m5_fault5=m5_fault5.reset_index(drop=True)
m5_fault5['TimeUS'] = pd.to_datetime(m5_fault5['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m5_fault5)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.173409,0.043199,0.004685,-0.176615,0.117010,-10.015199,0.07,0.62,-0.03,...,1.71,3173.0,1.92,1376.0,1364.0,1365.0,1375.0,1371.0,1370.0,1
1,00:00:00.002645,0.076037,-0.019434,-0.004237,-0.040051,-0.149500,-9.643477,0.07,0.62,-0.03,...,1.71,3173.0,1.92,1376.0,1364.0,1365.0,1375.0,1371.0,1370.0,1
2,00:00:00.005253,-0.153429,-0.056478,-0.003201,0.009088,-0.251848,-9.482471,0.07,0.62,-0.03,...,1.71,3173.0,1.92,1376.0,1364.0,1365.0,1375.0,1371.0,1370.0,1
3,00:00:00.008183,-0.206979,-0.032131,0.004984,-0.091977,-0.123594,-9.647797,0.07,0.62,-0.03,...,1.71,3173.0,1.92,1376.0,1364.0,1365.0,1375.0,1371.0,1370.0,1
4,00:00:00.011359,0.003761,0.014892,0.016421,-0.148840,0.052601,-10.130823,0.07,0.62,-0.03,...,1.71,3173.0,1.92,1376.0,1364.0,1365.0,1375.0,1371.0,1370.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18452,00:00:53.405210,0.344133,0.040225,-0.040295,-0.996638,0.970077,-13.997997,0.56,-0.34,-0.28,...,2.95,5482.0,2.33,1538.0,1435.0,1457.0,1520.0,1486.0,1492.0,1
18453,00:00:53.408244,0.342809,0.087223,-0.039791,-1.069123,1.056487,-14.131667,0.56,-0.34,-0.28,...,2.95,5482.0,2.33,1538.0,1435.0,1457.0,1520.0,1486.0,1492.0,1
18454,00:00:53.410800,0.355289,0.060848,-0.045159,-1.138574,0.975725,-14.337180,0.56,-0.34,-0.28,...,2.95,5482.0,2.33,1538.0,1435.0,1457.0,1520.0,1486.0,1492.0,1
18455,00:00:53.417201,0.368710,0.092210,-0.050912,-1.847238,0.714070,-15.517767,0.54,-0.07,-0.25,...,2.95,5482.0,2.33,1550.0,1372.0,1432.0,1501.0,1498.0,1436.0,1


## Analisi nel tempo e in frequenza

In [15]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [16]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})','guasto'])
    for i in range(int(len(m5_fault5)/350)):
        V1=m5_fault5[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2,'guasto':guasto}
        tab=tab.append(val,ignore_index=True)
    return tab

In [17]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
display(df_merged)

d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_14484\891270952.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_14484\891270952.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),guasto_x,...,guasto_y,<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),guasto
0,0.0,0.000470,0.011759,2.879044,0.108286,76.0,62.0,8.245381,7.829729,1.0,...,1.0,3.375755,0.688872,-0.792769,3.476008,102.0,98.0,15.085263,4.208971,1.0
1,1.0,0.000538,0.011727,2.894599,0.108140,82.0,65.0,8.186559,7.889835,1.0,...,1.0,3.378324,0.689232,-0.790849,3.478555,94.0,90.0,15.076279,4.167523,1.0
2,2.0,0.000633,0.011697,2.908780,0.108002,67.0,84.0,8.342994,8.124485,1.0,...,1.0,3.380878,0.689577,-0.788873,3.481086,100.0,96.0,15.066426,3.992749,1.0
3,3.0,0.000822,0.011677,2.914586,0.107908,68.0,86.0,8.495475,8.082675,1.0,...,1.0,3.383418,0.689908,-0.786841,3.483601,94.0,90.0,15.055513,3.696548,1.0
4,4.0,0.000940,0.011649,2.927193,0.107780,65.0,84.0,8.178334,8.003719,1.0,...,1.0,3.385944,0.690223,-0.784756,3.486100,94.0,92.0,15.043321,3.964772,1.0
5,5.0,0.001041,0.011619,2.940943,0.107647,84.0,66.0,7.906769,7.506176,1.0,...,1.0,3.388455,0.690524,-0.782616,3.488583,100.0,98.0,15.029602,4.610469,1.0
6,6.0,0.001159,0.011592,2.953533,0.107520,76.0,96.0,8.128664,7.803884,1.0,...,1.0,3.390952,0.690811,-0.780424,3.491051,94.0,92.0,15.014082,5.274932,1.0
7,7.0,0.001153,0.011559,2.970058,0.107370,69.0,88.0,8.465435,7.645683,1.0,...,1.0,3.393436,0.691084,-0.778179,3.493502,98.0,96.0,14.996462,5.947735,1.0
8,8.0,0.001176,0.011527,2.986649,0.107221,65.0,84.0,8.451355,7.486441,1.0,...,1.0,3.395905,0.691343,-0.775882,3.495939,96.0,94.0,14.976419,6.618464,1.0
9,9.0,0.001451,0.011522,2.979202,0.107202,74.0,92.0,8.012719,7.407520,1.0,...,1.0,3.398361,0.691588,-0.773534,3.498360,96.0,94.0,14.953613,7.276901,1.0


In [18]:
path_file = path_file.replace(r"M5\FAULT_M5_5", "")
os.chdir(path_file)
m5_fault5.to_csv('m5_fault5.csv', index=False)
print(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\
